In [21]:
import glob
import h5py
import numpy as np

Find data files recursively from root folder.

In [8]:
data_path = "D:/Coding/Thesis/Data/STFT Output/**/*.h5"
    
data_files = glob.glob(data_path, recursive=True)

print(data_files)

['D:/Coding/Thesis/Data/STFT Output\\DS99-08_05_25_04_19.h5', 'D:/Coding/Thesis/Data/STFT Output\\DS99-10_19_25_04_19.h5']


Load in data from h5 files.

Why is there \x0 in the name? Hex?

In [19]:
f = h5py.File(data_files[0], 'r')

['stft', 'taps']

Assign real and imaginary part of stft output separately to new numpy array.

In [23]:
df = np.empty(f['window_\x01']['stft']['real'].shape, 'complex64')

df.real = f['window_\x01']['stft']['real']
df.imag = f['window_\x01']['stft']['imag']